In [1]:
import pandas as pd
import re
import os

from pymongo import MongoClient
from utils import *

### Limpa Dataset de tipos de infrações

In [ ]:
df_tipos_infracoes_2018 = pd.read_excel("datasets/tipos-infracoes-2018.xlsx")
df_tipos_infracoes_2018 = df_tipos_infracoes_2018.rename(columns = {'Unnamed: 0': 'TIPO DE INFRACAO'})

df_tipos_infracoes_2018.head(n = 20)

### Limpa Dataset de acidentes com vítimas fatais

In [15]:
df_acidentes_fatais_2008_2017 = pd.read_excel("datasets/acidentes/acidentes-fatais-2008-2017.xlsx")
df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop("Fatal", axis = 1)
df_acidentes_fatais_2008_2017['Ano'] = 0


for ano in range(2008, 2018):
    df_acidentes_fatais_2008_2017.loc[df_acidentes_fatais_2008_2017['Ano ' + str(ano)] == 1.0, 'Ano'] = str(ano)
    df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop('Ano ' + str(ano), axis = 1)

df_acidentes_fatais_2008_2017.head()

,Região Administrativa,Nome da Via e Trecho,Descrição do Endereço,Natureza,Data,Dia da Semana,Hora,Faixa Horária (6x6),Nº de Vit Fatais,Nº de Vit Feridas,Veículos Múltiplos,Ano
0,ÁGUAS CLARAS,ALAMEDA DOS EUCALIPTOS,"ALAMEDA DOS EUCALIPTOS, QD 107, PRÓX. AO RESTA...",CHOQ OBJ FIXO,05/03/2015,5-QUI,20:00,18:00 - 23:59,1,-,MOTOCICLETA,2015
1,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, PRÓX. À UNIVERSIDADE CATÓLIC...",CAPOTAMENTO,11/05/2015,2-SEG,06:00,06:00 - 11:59,1,1,CAMINHONETA,2015
2,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO LOTE 48",COLISÃO,08/04/2011,6-SEX,18:10,18:00 - 23:59,1,-,CAMINHÃO x BICICLETA,2011
3,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO POSTO PO...",COLISÃO TRASEIRA,24/12/2009,5-QUI,11:40,06:00 - 11:59,1,-,ÔNIBUS x BICICLETA,2009
4,ÁGUAS CLARAS,AV. ARNIQUEIRA,"AV. ARNIQUEIRA, QD. 04, PRÓX. Á CHÁC. 81",ATROP PEDESTRE,17/11/2011,5-QUI,17:00,12:00 - 17:59,1,-,CAMINHONETA,2011


### Limpa Dataset de pedestres mortos em trechos não semaforizados

In [ ]:
df_pedestres_mortos_trechos_ns_2008_2017 = pd.read_excel("datasets/pedestres-mortos-trechos-ns-2008-2017.xlsx")

for ano in range(2008, 2018):
    df_pedestres_mortos_trechos_ns_2008_2017 = df_pedestres_mortos_trechos_ns_2008_2017.drop(ano, axis = 1)

df_pedestres_mortos_trechos_ns_2008_2017.head()

### Limpa Dataset de Cursos oferecidos pelo Detran

In [ ]:
df_cursos_oferecidos_detran_2018 = pd.read_excel("datasets/cursos_oferecidos_detran.xlsx")
df_cursos_oferecidos_detran_2018 = df_cursos_oferecidos_detran_2018.fillna(0)

for mes in df_cursos_oferecidos_detran_2018.columns[3:]:
    df_cursos_oferecidos_detran_2018[mes] = df_cursos_oferecidos_detran_2018[mes].astype(int)

df_cursos_oferecidos_detran_2018.head()

### Limpa Dataset de campanhas oferecidas pelo Detran

In [20]:
df_campanhas_educativas_2012_2019 = pd.read_excel("datasets/campanhas_educativas_2012_2019.xlsx")

for column in df_campanhas_educativas_2012_2019.columns[6:10]:
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].fillna(0)
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].astype(int)

df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'] = df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'].fillna("--")

df_campanhas_educativas_2012_2019.head()

,Campanha,Objetivos da Campanha,Data de Início da campanha,Data Final da Campanha,Local do Evento da Campanha,Região Administrativa do Evento da Campanha,Qtd de Motoristas Abordados no Evento,Qtd de Pedestres Abordados no Evento,Qtd De Motociclistas Abordados no Evento,Qtd de Ciclistas Abordados no Evento,Data Inicial do Evento da Campanha,Hora Inicial do Evento da Campanha,Data Final do Evento da Campanha,Hora Final do Evento da Campanha,Relatório do Evento,Rodovia
0,GECED - BLITZEN NOTURNAS,Campanha Educativa de entrega de folder Alcool...,2015-11-27 00:00:00.000,2015-10-27 00:00:00.000,BR 070,--,700,0,0,0,2012-01-06,00:00:00:00,2012-01-06,00:00:00:00,NaN,NaN
1,Operação Funil,"Álcool e direção, conscientização de motoristas.",2012-01-10 00:00:00.000,2012-01-10 00:00:00.000,BR 040 - Posto da Polícia Rodoviária Federal,--,300,0,0,0,2012-01-10,00:00:00:00,2012-01-10,00:00:00:00,NaN,NaN
2,Operação Funil,Abordagem de carros. Operação iniciada ás 15:3...,2012-01-12 00:00:00.000,2012-01-12 00:00:00.000,BR 251/ DF 001 - Saída para Unaí,--,448,0,0,0,2012-01-12,00:00:00:00,2012-01-12,00:00:00:00,NaN,NaN
3,Operação Funil,Abordagem de carros iniciando ás 15h e finaliz...,2012-01-19 00:00:00.000,2012-12-30 00:00:00.000,CPRV BR 060 - Samambaia,--,550,0,0,0,2012-01-19,00:00:00:00,2012-01-19,00:00:00:00,NaN,NaN
4,Operação Funil,Abordagem de carros.,2012-01-06 00:00:00.000,2012-12-30 00:00:00.000,BR 020 Posto da BPRv,--,0,0,0,0,2012-01-20,00:00:00:00,2012-01-20,00:00:00:00,NaN,NaN


### Limpa Dataset de acidentes
* Um Carregamento por vez devido ao uso de memória

In [42]:
rootdir = 'datasets/acidentes/ocorrencias'

def extrai_acidentes_rodovias():
    
    df_acidentes_rodovias = pd.DataFrame()
    
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            
            filepath = os.path.join(subdir, file)
            ano = filepath.split('rodovias-')[1].split('.xlsx')[0]
            
            print(filepath)
            
            if ano == '2017' or ano == '2018' or ano == '2019':
                columns_to_drop = ['id', 'municipio', 'latitude', 'longitude', 'regional', 'delegacia', 'uop']
            else:
                columns_to_drop = ['id', 'municipio']
                
            
            df_temp = pd.read_excel(filepath)
            df_temp = df_temp[df_temp.uf == 'DF']
            df_temp['data_inversa'] = pd.to_datetime(df_temp['data_inversa'])
            df_temp['uf'] = df_temp['uf'].fillna("")
            df_temp = df_temp.drop(columns = columns_to_drop, axis = 1)
        
            if ano =='2016' or ano == '2017' or ano == '2018' or ano == '2019':
                df_temp['ano'] = df_temp['data_inversa'].dt.year
            
            df_temp = df_temp[['data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'ano', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos']]
            df_acidentes_rodovias = pd.concat([df_acidentes_rodovias, df_temp])
    
    return df_acidentes_rodovias          

In [43]:
testzao = extrai_acidentes_rodovias()
testzao.info()

datasets/acidentes/ocorrencias/acidentes-rodovias-2018.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2012.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2016.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2010.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2014.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2013.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2019.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2015.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2017.xlsx
datasets/acidentes/ocorrencias/acidentes-rodovias-2011.xlsx
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11811 entries, 8 to 192077
Data columns (total 24 columns):
data_inversa              11811 non-null datetime64[ns]
dia_semana                11811 non-null object
horario                   11811 non-null object
uf                        11811 non-null object
br                        11806 non-null object
km                        11806 non-null o

### Limpa Dataset de infrações DER

In [2]:
for filename in os.listdir('datasets/infracoes/DER/'): 
    dst = filename.replace("_", "-")
    src = 'datasets/infracoes/DER/' + filename 
    dst = 'datasets/infracoes/DER/' + dst 
    os.rename(src, dst) 

In [13]:
df_infracoes_der_2018_2019 = pd.DataFrame()
path = 'datasets/infracoes/DER/'

for filename in os.listdir(path):
    print("Current File: " + path + filename)
    
    abs_path = path + filename
    df_temp = pd.read_excel(abs_path)
    
    df_infracoes_der_2018_2019 = pd.concat([df_infracoes_der_2018_2019, df_temp])

df_infracoes_der_2018_2019.head()


Current File: datasets/infracoes/DER/2019-jan-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-mar-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2018-dez-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-jul-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-abr-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2018-nov-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-fev-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-jun-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2018-out-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2019-mai-autosinfrac.xlsx
Current File: datasets/infracoes/DER/2018-ago-autosinfrac.xlsx


,tipo_infracao,descricao,tipo_infrator,tipo_veiculo,cometimento,hora_cometimento,auinf_local_rodovia,auinf_local_km,auinf_local_referencia,auinf_local_complemento,auinf_local_latitude,auinf_local_longitude,grav_tipo
0,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,UTILITARIO,01/01/2019,00:02,"DF-025 (EPDB) KM 24,5 SENTIDO PARANOA A PONTE ...",NaN,NaN,NaN,NaN,NaN,Média
1,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,AUTOMOVEL,01/01/2019,00:02,"DF-079 (EPVP) KM 3,6  SENTIDO DF-075 (EPNB) ...",NaN,NaN,NaN,NaN,NaN,Gravíssima
2,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,CAMINHAO,01/01/2019,00:06,"DF-001 (EPCT) KM 63,5 SENTIDO SAMAMBAIA ...",NaN,NaN,NaN,NaN,NaN,Média
3,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,CAMIONETA,01/01/2019,00:09,"DF-085 (EPTG) KM 9,0 SENTIDO DF-003(PLANO PILO...",NaN,NaN,NaN,NaN,NaN,Média
4,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,AUTOMOVEL,01/01/2019,00:11,"DF-005(EPPR)Km 1,5 Sent. DF-006/Varjão ...",NaN,NaN,NaN,NaN,NaN,Média


In [22]:
df_infracoes_der_2018_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817012 entries, 0 to 61892
Data columns (total 13 columns):
tipo_infracao              817012 non-null object
descricao                  817012 non-null object
tipo_infrator              817012 non-null object
tipo_veiculo               807773 non-null object
cometimento                817012 non-null object
hora_cometimento           817012 non-null object
auinf_local_rodovia        817012 non-null object
auinf_local_km             258245 non-null object
auinf_local_referencia     272541 non-null object
auinf_local_complemento    90351 non-null object
auinf_local_latitude       259428 non-null object
auinf_local_longitude      259428 non-null object
grav_tipo                  817012 non-null object
dtypes: object(13)
memory usage: 87.3+ MB


### Salva datasets de número de habilitados e frota de veículos

In [49]:
df_numero_habilitados_2009_2018 = pd.read_excel("datasets/numero-habilitados-df-2009-2018.xlsx")
df_numero_habilitados_2009_2018.columns = ['tipo_habilitacao', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
cols = list(df_numero_habilitados_2009_2018.columns.values)
cols = cols[::-1]
df_numero_habilitados_2009_2018 = df_numero_habilitados_2009_2018[cols]

df_frota_veiculos_2009_2019 = pd.read_excel("datasets/frota-veiculos-df-2009-2019.xlsx")
df_frota_veiculos_2009_2019.columns = ['tipo', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

### Cria Dataset de dados demográficos de Brasília

In [12]:
df_densidade_urbana_2015 = pd.read_excel("datasets/densidade-urbana-2015.xlsx")
df_distancia_centro_2015 = pd.read_excel("datasets/distancia-plano-piloto.xlsx")

df_demografia_brasilia_2015_2016 = pd.merge(df_densidade_urbana_2015, df_distancia_centro_2015, left_on = 'RA', right_on = 'RA')
df_demografia_brasilia_2015_2016.head()

,RA,População 2013,População 2015/2016,TMGCA (%),Área com Ocupação Urbana (em ha),Densidade Urbana 2015,Área Total da RA (em ha),Densidade Demográfica 2015,Casas (%),Apart + Quit (%),Distância do Centro (Plano Piloto) em Km,Grupo de Renda,Renda Per Capita 2015
0,RA XXIII - Varjão,9292,8453,-4.62,59.42,142.27,75.56,111.87,75.75,20.44,11,Baixa,627.81
1,RA XIX - Candangolândia,16886,15641,-3.76,113.85,137.38,662.70,23.60,87.20,12.00,11,Média Baixa Renda,1460.98
2,RA IX - Ceilândia,451872,479713,3.03,3691.92,129.94,23401.14,20.50,94.36,4.25,26,Média Baixa Renda,914.61
3,RA XV - Recanto das Emas,138997,146906,2.81,1246.27,117.88,10261.11,14.32,96.98,2.76,26,Média Baixa Renda,803.92
4,RA XIV - São Sebastião,98908,99525,0.31,882.69,112.75,35571.37,2.80,92.71,6.61,26,Média Baixa Renda,966.96


### Salva Datasets de infrações (PRF)

In [52]:
rootdir = 'datasets/infracoes/PRF'

def verifica_colunas():
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = os.path.join(subdir, file)
            print(filepath)
            df_temp = pd.read_csv(filepath, sep = ',', encoding = 'utf8')
            
            if 'hora' not in df_temp.columns:
                df_temp['hora'] = 'NAO_ESPECIFICADA'
                
            columns = ['dat_infracao', 'tip_abordagem', 'ind_assinou_auto', 'ind_veiculo_estrangeiro', 'ind_sentido_trafego', 'uf_placa', 'uf_infracao', 'num_br_infracao', 'num_km_infracao', 'nom_municipio', 'descricao_abreviada', 'enquadramento', 'data_inicio_vigencia', 'data_fim_vigencia', 'med_realizada', 'med_considerada', 'exc_verificado', 'especie', 'nome_veiculo_marca', 'nom_modelo_veiculo', 'hora']
            columns_current = list(df_temp.columns.values)

            if columns == columns_current:
                print(True)
            else:
                print(False)

                
def extrai_infracoes():
    
    infracoes = pd.DataFrame()
    
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            
            filepath = os.path.join(subdir, file)
            print(filepath)
            
            ano = subdir.split('PRF/')[1]
            if ano == '2017' or ano == '2018':
                file_encoding = 'iso-8859-1'
            else:
                file_encoding = 'utf8'
             
            df_temp = pd.read_csv(filepath, sep = ',', encoding = file_encoding)
            df_temp = df_temp[df_temp.uf_infracao == 'DF']
            
            if 'hora' not in df_temp.columns:
                df_temp['hora'] = 'NAO_ESPECIFICADA'
            
            if 'cod_infracao' in df_temp.columns:
                df_temp = df_temp.drop("cod_infracao", axis = 1)
            
            infracoes = pd.concat([infracoes, df_temp])
    
    return infracoes

In [3]:
df_infracoes_2010_2018 = extrai_infracoes()

datasets/infracoes/PRF/2013/jan_fev.csv
datasets/infracoes/PRF/2013/set_out.csv
datasets/infracoes/PRF/2013/mar_abr.csv
datasets/infracoes/PRF/2013/jul_ago.csv
datasets/infracoes/PRF/2013/mai_jun.csv
datasets/infracoes/PRF/2013/nov_dez.csv
datasets/infracoes/PRF/2014/jan_fev.csv
datasets/infracoes/PRF/2014/set_out.csv
datasets/infracoes/PRF/2014/mar_abr.csv
datasets/infracoes/PRF/2014/jul_ago.csv
datasets/infracoes/PRF/2014/mai_jun.csv
datasets/infracoes/PRF/2014/nov_dez.csv
datasets/infracoes/PRF/2012/jan_fev.csv
datasets/infracoes/PRF/2012/set_out.csv
datasets/infracoes/PRF/2012/mar_abr.csv
datasets/infracoes/PRF/2012/jul_ago.csv
datasets/infracoes/PRF/2012/mai_jun.csv
datasets/infracoes/PRF/2012/nov_dez.csv
datasets/infracoes/PRF/2016/jun.csv
datasets/infracoes/PRF/2016/set.csv
datasets/infracoes/PRF/2016/abr.csv
datasets/infracoes/PRF/2016/out.csv
datasets/infracoes/PRF/2016/ago.csv
datasets/infracoes/PRF/2016/jan.csv
datasets/infracoes/PRF/2016/jul.csv
datasets/infracoes/PRF/2016/

/home/iago/anaconda3/envs/tccEnv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


datasets/infracoes/PRF/2016/dez.csv
datasets/infracoes/PRF/2016/mar.csv
datasets/infracoes/PRF/2016/fev.csv
datasets/infracoes/PRF/2015/jun.csv
datasets/infracoes/PRF/2015/set.csv
datasets/infracoes/PRF/2015/abr.csv
datasets/infracoes/PRF/2015/out.csv
datasets/infracoes/PRF/2015/ago.csv
datasets/infracoes/PRF/2015/jan.csv
datasets/infracoes/PRF/2015/jul.csv
datasets/infracoes/PRF/2015/mai.csv
datasets/infracoes/PRF/2015/nov.csv
datasets/infracoes/PRF/2015/dez.csv
datasets/infracoes/PRF/2015/mar.csv
datasets/infracoes/PRF/2015/fev.csv
datasets/infracoes/PRF/2018/jun.csv
datasets/infracoes/PRF/2018/set.csv
datasets/infracoes/PRF/2018/abr.csv


/home/iago/anaconda3/envs/tccEnv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


datasets/infracoes/PRF/2018/out.csv
datasets/infracoes/PRF/2018/ago.csv
datasets/infracoes/PRF/2018/jan.csv
datasets/infracoes/PRF/2018/jul.csv
datasets/infracoes/PRF/2018/mai.csv
datasets/infracoes/PRF/2018/nov.csv
datasets/infracoes/PRF/2018/dez.csv
datasets/infracoes/PRF/2018/mar.csv
datasets/infracoes/PRF/2018/fev.csv
datasets/infracoes/PRF/2010/jan_fev.csv
datasets/infracoes/PRF/2010/set_out.csv
datasets/infracoes/PRF/2010/mar_abr.csv
datasets/infracoes/PRF/2010/jul_ago.csv
datasets/infracoes/PRF/2010/mai_jun.csv
datasets/infracoes/PRF/2010/nov_dez.csv
datasets/infracoes/PRF/2017/jun.csv
datasets/infracoes/PRF/2017/set.csv
datasets/infracoes/PRF/2017/abr.csv
datasets/infracoes/PRF/2017/out.csv
datasets/infracoes/PRF/2017/ago.csv
datasets/infracoes/PRF/2017/jan.csv
datasets/infracoes/PRF/2017/jul.csv
datasets/infracoes/PRF/2017/mai.csv
datasets/infracoes/PRF/2017/nov.csv
datasets/infracoes/PRF/2017/dez.csv
datasets/infracoes/PRF/2017/mar.csv
datasets/infracoes/PRF/2017/fev.csv
data

#### Remove colunas desnecessárias

In [96]:
df_infracoes_2010_2018 = infracoes_2010_2018.drop("ind_assinou_auto", axis = 1)
df_infracoes_2010_2018 = infracoes_2010_2018.drop("nom_municipio", axis = 1)
df_infracoes_2010_2018 = infracoes_2010_2018.drop("data_fim_vigencia", axis = 1)
df_infracoes_2010_2018 = infracoes_2010_2018.drop("data_inicio_vigencia", axis = 1)
df_infracoes_2010_2018 = infracoes_2010_2018.drop("enquadramento", axis = 1)

In [97]:
df_infracoes_2010_2018

,dat_infracao,tip_abordagem,ind_veiculo_estrangeiro,ind_sentido_trafego,uf_placa,uf_infracao,num_br_infracao,num_km_infracao,descricao_abreviada,med_realizada,med_considerada,exc_verificado,especie,nome_veiculo_marca,nom_modelo_veiculo,hora
62,2013-01-01,C,N,C,DF,DF,40,0,Dirigir sob a influência de álcool,NaN,NaN,NaN,PASSAGEIRO,RENAULT,RENAULT/LOGAN AU,NAO_ESPECIFICADA
88,2013-01-01,C,N,C,DF,DF,40,0,Dirigir veículo sem possuir CNH ou Permissão p...,NaN,NaN,NaN,MISTO,VW,VW/GOL MI,NAO_ESPECIFICADA
89,2013-01-01,C,N,C,DF,DF,40,0,Conduzir veículo sem os documentos de porte ob...,NaN,NaN,NaN,MISTO,VW,VW/GOL MI,NAO_ESPECIFICADA
114,2013-01-01,C,N,C,DF,DF,40,0,Conduzir o veículo registrado que não esteja d...,NaN,NaN,NaN,PASSAGEIRO,FIAT,FIAT SIENA ELX F,NAO_ESPECIFICADA
115,2013-01-01,C,N,C,DF,DF,40,0,Dirigir veículo com validade da CNH ou PPD ven...,NaN,NaN,NaN,PASSAGEIRO,FIAT,FIAT SIENA ELX F,NAO_ESPECIFICADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463332,2011-12-31,C,N,C,GO,DF,40,0,Conduzir o veículo sem equipamento obrigatório,NaN,NaN,NaN,CARGA,VW,8.150,NAO_ESPECIFICADA
463609,2011-12-31,C,N,C,DF,DF,40,0,Conduzir o veículo com equip obrigatório em de...,NaN,NaN,NaN,CARGA,VW,8.140,NAO_ESPECIFICADA
466717,2011-12-31,C,N,C,DF,DF,40,4,Dirigir sem atenção ou sem os cuidados indispe...,NaN,NaN,NaN,PASSAGEIRO,GM,CORSA,NAO_ESPECIFICADA
467001,2011-12-31,C,N,D,DF,DF,60,18,Dirigir sob a influência de álcool,NaN,NaN,NaN,ESPECIAL,FIAT,FIAT/STRADA ADVE,NAO_ESPECIFICADA


### Refresh Database

In [99]:
# Deletes all records, only if necessary
clear_database()

### Remove registros duplicados das collections

In [10]:
df_acidentes_fatais_2008_2017.drop_duplicates(keep = False, inplace = True)

In [21]:
df_campanhas_educativas_2012_2019.drop_duplicates(keep = False, inplace = True)

In [46]:
df_acidentes_rodovias_2010_2019.drop_duplicates(keep = False, inplace = True)

In [58]:
df_infracoes_2010_2018_1st_half = df_infracoes_2010_2018[df_infracoes_2010_2018.hora == 'NAO_ESPECIFICADA']
df_infracoes_2010_2018_2nd_half = df_infracoes_2010_2018[df_infracoes_2010_2018.hora != 'NAO_ESPECIFICADA']

df_infracoes_2010_2018_2nd_half.drop_duplicates(keep = False, inplace = True)

df_infracoes_2010_2018 = pd.concat([df_infracoes_2010_2018_1st_half, df_infracoes_2010_2018_2nd_half])

/home/iago/anaconda3/envs/tccEnv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Salvar e recuperar DataFrames

In [ ]:
dataframe_to_mongo(df_tipos_infracoes_2018, "tipos_infracoes_2018")

In [ ]:
dataframe_to_mongo(df_acidentes_fatais_2008_2017, "acidentes_fatais_2008_2017")

In [ ]:
dataframe_to_mongo(df_pedestres_mortos_trechos_ns_2008_2017, "pedestres_mortos_trechos_ns_2008_2017")

In [ ]:
dataframe_to_mongo(df_cursos_oferecidos_detran_2018, "cursos_oferecidos_detran_2018")

In [ ]:
dataframe_to_mongo(df_campanhas_educativas_2012_2019, "campanhas_educativas_detran_2012_2019")

In [ ]:
dataframe_to_mongo(df_acidentes_rodovias_2010_2019, "acidentes_rodovias_2010_2019")

In [ ]:
dataframe_to_mongo(df_demografia_brasilia_2015_2016, "demografia_brasilia_2015_2016")

In [ ]:
dataframe_to_mongo(df_infracoes_der_2018_2019, "infracoes_der_2018_2019")

In [ ]:
dataframe_to_mongo(df_numero_habilitados_2009_2018, "numero_habilitados_2009_2018")

In [ ]:
dataframe_to_mongo(df_frota_veiculos_2009_2019, "frota_veiculos_2009_2019")

In [62]:
dataframe_to_mongo(df_infracoes_2010_2018, "infracoes_prf_2010_2018")